## 自定义输出
 - 输出函数参数
 - 输出 json
 - 输出 List
 - 输出 日期等等

### 讲笑话机器人 
- 希望每次根据指令，可以输出一个这样的笑话：小王怎么死的？蠢死的。

In [21]:
from langchain_ollama import OllamaLLM
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List

llm = OllamaLLM(
    model="llama3.1:8b",
    base_url="http://localhost:11434",
)

class JokeResponse(BaseModel):
    setup: str = Field(description="笑话的问题")
    punchline: str = Field(description="笑话的答案")

    # model_json_schema = {
    #     "type": "object",
    #     "properties": {
    #         "setup": {"type": "string"},
    #         "punchline": {"type": "string"}
    #     },
    #     "required": ["setup", "punchline"]
    # }

    @validator("setup")
    def check_setup(cls, field: Field):
        if field.value[-1] != "?":
            raise ValueError("问题必须以问号结尾")
        return field.value

parser = PydanticOutputParser(pydantic_object=JokeResponse)

pt = PromptTemplate(
    template="回答用户的输入.\n{format_instruction}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instruction": parser.get_format_instructions()},
)

prompt_amd_model = pt | llm
out_put = prompt_amd_model.invoke({"query": "给我讲一个笑话"})
print("输出结果:\n", out_put)

# 验证解析结果
parsed_output = parser.invoke(out_put)
print("解析后的结构:\n", parsed_output)

AttributeError: type object 'JokeResponse' has no attribute 'model_json_schema'